In [2]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=400b47ce7b7eb3b78c00d695ce4afa6ac8626063a820ba40a3bf92672dbf5aab
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [3]:
import feedparser
import csv
from datetime import datetime
import re

# NOTE: some text handling nay be needed afterwards (can't take them all out...)
# change csv file name if needed

# Python function: scrap news article
class googleNewsFeedScraper:

    # initialize class to take different arguments' values
    def __init__(self, query, start_date, end_date, file_name):
        self.query = query
        self.start_date = start_date
        self.end_date = end_date
        self.file_name=file_name

    # clean the title with string formatting
    def clean_title(self, title):
        """Cleans the title by replacing unwanted characters."""

        # try-except block incase encounter error
        try:
            # encode and decode uncommon characters to detect anomalies (NLP)
            title = title.encode('utf-8').decode('utf-8')
            title = re.sub(r'‘', "'", title)
            title = re.sub(r'’', "'", title)
            title = re.sub(r'“', '"', title)
            title = re.sub(r'”', '"', title)
            title = re.sub(r'â€[^\s]*', "'", title)
            title = re.sub("Â", "", title)
            title = re.sub("â€™", "'", title)
            title = re.sub("â€œ", '"', title)
            title = re.sub('â€“', '-', title)
            title = re.sub('â€', '"', title)
        except Exception as e:
            print(f"Encoding error for title: {title}. Error: {e}")

        # return output
        return title.strip()

    # Main function for all the work
    def scrape_google_news_feed(self):
        # locate url
        rss_url = f'https://news.google.com/rss/search?q={self.query}+after:{self.start_date}+before:{self.end_date}&hl=en-CA&gl=CA&ceid=CA:en'
        print(rss_url)
        feed = feedparser.parse(rss_url)

        # Create or open a CSV file for writing
        with open(self.file_name, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            # Write the header for csv
            writer.writerow(['Title', 'News Agency', 'Link', 'Date'])

            # locate info
            if feed.entries:
                for entry in feed.entries:
                    # gather info
                    full_title = entry.title
                    link = entry.link
                    pubdate = entry.published

                    # Clean the title by calling the function
                    full_title = self.clean_title(full_title)

                    # Format the publication date
                    formatted_date = datetime.strptime(pubdate, '%a, %d %b %Y %H:%M:%S %Z').strftime('%d-%m-%Y')

                    # Split the title to get the news agency
                    if '- ' in full_title:
                        title, news_agency = full_title.split('- ', 1)
                    else:
                        title, news_agency = full_title, 'Unknown'

                    # Write the entry to the CSV file
                    writer.writerow([title.strip(), news_agency.strip(), link, formatted_date])

                    # Print to console for confirmation (OPTIONAL)
                    # print(f"Title: {title}\nLink: {link}\nDate: {formatted_date}\n")
                    # print("-+-")
            else:
                print("Nothing Found!")

# declare instances here
if __name__ == "__main__":
    query = 'raygun'
    start_date = '2024-08-01'
    end_date = '2024-09-01'
    file_name= 'news_feed_temp.csv'

    scraper = googleNewsFeedScraper(query, start_date, end_date, file_name)
    scraper.scrape_google_news_feed()


https://news.google.com/rss/search?q=raygun+after:2024-08-01+before:2024-09-01&hl=en-CA&gl=CA&ceid=CA:en


In [4]:
# reading csv file for testing

import pandas as pd

df=pd.read_csv("/content/news_feed_temp.csv")

In [5]:
df

,Title,News Agency,Link,Date
0,Raygun Olympics: How viral breaker made it to ...,BBC.com,https://news.google.com/rss/articles/CBMiWkFVX...,17-08-2024
1,Australian B-Girl Raygun 'devastated' by onlin...,CBC Sports,https://news.google.com/rss/articles/CBMihAFBV...,15-08-2024
2,"The 36-Year-Old Breakdancer Named ""Raygun"" Res...",Yahoo News Canada,https://news.google.com/rss/articles/CBMigAFBV...,10-08-2024
3,"Raygun earned her spot in the Olympics. Yes, r...",Vox.com,https://news.google.com/rss/articles/CBMiiAFBV...,14-08-2024
4,I Just Found Out Viral Breakdancer Raygun Got ...,CinemaBlend,https://news.google.com/rss/articles/CBMinAFBV...,26-08-2024
...,...,...,...,...
85,Did RayGun actually score zero points at the O...,indy100,https://news.google.com/rss/articles/CBMiaEFVX...,16-08-2024
86,Raygun: Olympic judge praises breaker's 'origi...,BBC.com,https://news.google.com/rss/articles/CBMiWkFVX...,11-08-2024
87,"""Raygun"" Gunn rigged her way into Paris 2024: ...",Marca English,https://news.google.com/rss/articles/CBMikgFBV...,13-08-2024
88,Rachel Dratch Spoofs Olympic Breakdancer Raygu...,Variety,https://news.google.com/rss/articles/CBMisAFBV...,13-08-2024
